In [40]:
#set openai key in .env.local
import os
import pandas as pd
import openai
import numpy as np
import tensorflow as tf
import regex as re
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding

#from openai.embeddings_utils import get_embedding

In [41]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'no_product_variants.pkl')
product_data = product_data.reset_index(drop=True)
# product_data = product_data.sample(n=10, random_state=2).reset_index(drop=True)
display(product_data)

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,40.0,schwarz,02.643.57,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0
1,551622,4054452768212,elegante Pumps Glattleder schwarz,38.0,schwarz,05.160.37,6590677,315573,w,PT,...,Pumps,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0
2,547193,4054452768427,Slipper Glattleder schwarz,42.0,schwarz,04.443.27,6590677,315576,w,SK,...,Halbschuhe,Damenschuhe,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0
3,549776,4059701687894,Sneaker low Rauleder blau,39.0,blau,06.968.46,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,37.5,schwarz,06.102.67,6590678,315574,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,35.0,blauw,24.764.36,6590677,315568,w,SK,...,Sandalen,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,40.5,pink,26.090.21,6590678,315590,w,PT,...,Halbschuhe,Damenschuhe,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0
3294,10782704,4251234499207,Shopper ANDIE blau,,blau,921453,363013,363017,w,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3295,10679703,4066558303617,Elegante pumps Glad leer wit,35.0,wit,21.450.60,6590677,315573,w,PT,...,Pumps,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0


In [42]:
def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def combine_columns(row):
    return preprocess_text (
            "ID: " + str(row.product__id) + "; " +
            "product_number: " + str(row.productNumber) + "; " +
            "product_name: " + row.productName + "; " +
            # "size: " + str(row.productSizeEu) + "; " +
            "color: " + str(row.productColorName) + "; " +
            "article number: " + str(row.articleNumber) + "; " +
            "brand: " + str(row.brand__id) + "; " +
            "main category: " + str(row.mainCategory__id) + "; " +
            "gender: " + str(row.gender) + "; " +
            "line: " + str(row.line) + "; " +
            "price: " + str(row.b2cPrice) + "; " +
            "material: " + str(row.material) + "; " +
            "material PM: " + str(row.materialPm) + "; "
    )


product_data_embeddings = product_data[['product__id']].copy()
product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
display(product_data_embeddings)

,product__id,combined
0,557559,id 557559 product_number 4058394021466 product...
1,551622,id 551622 product_number 4054452768212 product...
2,547193,id 547193 product_number 4054452768427 product...
3,549776,id 549776 product_number 4059701687894 product...
4,550726,id 550726 product_number 4054452851594 product...
...,...,...
3292,10603553,id 10603553 product_number 4066558951733 produ...
3293,10442948,id 10442948 product_number 4065171827272 produ...
3294,10782704,id 10782704 product_number 4251234499207 produ...
3295,10679703,id 10679703 product_number 4066558303617 produ...


In [43]:
vectorizer = TextVectorization(output_mode='int', output_sequence_length=10)

# Apply the custom preprocessing function to the 'comb
# Update the vocabulary based on the preprocessed text
vectorizer.adapt(product_data_embeddings['combined'].values)

In [44]:
import os
import requests
import zipfile
import numpy as np

def download_and_extract_glove(glove_url, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    glove_zip = os.path.join(save_dir, 'glove.6B.zip')
    glove_txt = os.path.join(save_dir, 'glove.6B.100d.txt')

    if not os.path.exists(glove_txt):
        print("Downloading GloVe embeddings...")
        response = requests.get(glove_url, stream=True)
        with open(glove_zip, 'wb') as f:
            f.write(response.content)

        print("Extracting GloVe embeddings...")
        with zipfile.ZipFile(glove_zip, 'r') as zip_ref:
            zip_ref.extractall(save_dir)

        os.remove(glove_zip)

# Replace 'save_directory' with the path to the directory where you want to save the embeddings
save_directory = '.'
glove_url = 'https://nlp.stanford.edu/data/glove.6B.zip'
download_and_extract_glove(glove_url, save_directory)

In [45]:
glove_txt = os.path.join(save_directory, 'glove.6B.100d.txt')
embeddings_index = {}
with open(glove_txt, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Define the Embedding layer using the pre-trained GloVe embeddings
vocab_size = len(vectorizer.get_vocabulary())
embedding_dim = 100  # You can choose the desired embedding dimension
embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = {word: index for index, word in enumerate(vectorizer.get_vocabulary())}

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False
)



In [46]:
# Convert the preprocessed text to numeric vectors
numeric_vectors = vectorizer(product_data_embeddings['combined'].values)

# Get word embeddings
word_embeddings = embedding_layer(numeric_vectors)
word_embeddings, word_embeddings.shape

(<tf.Tensor: shape=(3297, 10, 100), dtype=float32, numpy=
 array([[[-0.22458  , -0.58967  ,  0.43007  , ..., -0.63613  ,
           0.31284  , -0.82294  ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.53366  ,  0.0085679, -0.85512  , ...,  0.21118  ,
           0.22603  ,  0.18089  ],
         [-0.52516  ,  0.44118  , -0.1273   , ..., -0.88581  ,
           0.075446 ,  0.040447 ]],
 
        [[-0.22458  , -0.58967  ,  0.43007  , ..., -0.63613  ,
           0.31284  , -0.82294  ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.    

In [47]:
# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
from sklearn.metrics.pairwise import cosine_similarity

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    input_embedding_flat = query_embedding.numpy().reshape(1, -1)
    word_embeddings_flat = neighbors_embedding.numpy().reshape(word_embeddings.shape[0], -1)

    # Calculate the cosine similarity between the input embedding and all other word embeddings
    similarities = cosine_similarity(input_embedding_flat, word_embeddings_flat)
    # Get the indices of the most similar neighbors
    neighbor_indices = np.argsort(similarities[0])[::-1][:n]

    scores = [similarities[0][i] for i in neighbor_indices]
   
    return neighbor_indices, scores



indices = get_n_nearest_neighbors(word_embeddings[0], word_embeddings)
display(indices)

(array([  0,   8, 298,  35,  50]), [1.0, 1.0, 0.9378928, 0.9378928, 0.9378928])

In [48]:
product_data_embeddings.iloc[indices].combined

,combined,combined
0,id 557559 product_number 4058394021466 product...,id 557559 product_number 4058394021466 product...
8,id 548618 product_number 4058394180835 product...,id 548618 product_number 4058394180835 product...
298,id 3844889 product_number 4060666972710 produc...,id 3844889 product_number 4060666972710 produc...
35,id 549918 product_number 4054452741932 product...,id 549918 product_number 4054452741932 product...
50,id 407474 product_number 4054452837635 product...,id 407474 product_number 4054452837635 product...


In [49]:
product_data.iloc[indices]

,productNumber,productNumber,product__id,product__id,product__id
0,4058394021466,4058394021466,557559,557559,557559
8,4058394180835,4058394180835,548618,548618,548618
298,4060666972710,4060666972710,3844889,3844889,3844889
35,4054452741932,4054452741932,549918,549918,549918
50,4054452837635,4054452837635,407474,407474,407474


In [64]:

def get_best_recommendations_multiple(query_embeddings, neighbors_embeddings, n = 5):
    all_results = {}
    for query_embedding in query_embeddings:
        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)
        result_dict = {index: score for index, score in zip(indices, scores)}
        for key, value in result_dict.items():
            if key in all_results:
                all_results[key] = max(all_results[key], value)
            else: 
                all_results[key] = value
    return dict(sorted(all_results.items(), key=lambda item: item[1], reverse=True)[:n])


best_items = get_best_recommendations_multiple([word_embeddings[0], word_embeddings[1]], word_embeddings)
            
print(best_items)

{0: 1.0, 8: 1.0, 1: 0.99999994, 298: 0.9378928, 35: 0.9378928}
